## 准备数据

In [23]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [24]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [26]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([784, 256]))  # 输入层到隐藏层的权重
        self.b1 = tf.Variable(tf.zeros([256]))              # 隐藏层的偏置
        self.W2 = tf.Variable(tf.random.normal([256, 10]))  # 隐藏层到输出层的权重
        self.b2 = tf.Variable(tf.zeros([10]))               # 输出层的偏置
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        
        x = tf.reshape(x, [-1, 784])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2

        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [27]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [28]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 140.68106 ; accuracy 0.07215
epoch 1 : loss 129.05286 ; accuracy 0.08121666
epoch 2 : loss 120.66511 ; accuracy 0.09071667
epoch 3 : loss 114.16687 ; accuracy 0.10095
epoch 4 : loss 108.9299 ; accuracy 0.10981666
epoch 5 : loss 104.56452 ; accuracy 0.11835
epoch 6 : loss 100.79435 ; accuracy 0.12518333
epoch 7 : loss 97.43073 ; accuracy 0.13198334
epoch 8 : loss 94.35347 ; accuracy 0.13776666
epoch 9 : loss 91.48104 ; accuracy 0.14366667
epoch 10 : loss 88.7773 ; accuracy 0.14903334
epoch 11 : loss 86.22116 ; accuracy 0.1548
epoch 12 : loss 83.80877 ; accuracy 0.16075
epoch 13 : loss 81.539795 ; accuracy 0.16708334
epoch 14 : loss 79.40898 ; accuracy 0.17413333
epoch 15 : loss 77.414 ; accuracy 0.1824
epoch 16 : loss 75.542725 ; accuracy 0.18958333
epoch 17 : loss 73.76967 ; accuracy 0.1972
epoch 18 : loss 72.08442 ; accuracy 0.20441666
epoch 19 : loss 70.47698 ; accuracy 0.21178333
epoch 20 : loss 68.938995 ; accuracy 0.21931666
epoch 21 : loss 67.4628 ; accuracy 0.2263